In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
from numpy import argmax
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
from pandas import read_excel
df = read_excel('input_dumps1_1.xlsx', sheet_name = 'Sheet1' )
#print(df.head()) # shows headers with top 5 rows

In [72]:
fields = ['Short description', 'Assignment group']
df = pd.read_excel('user_incidents_dumps_60days.xlsx', skipinitialspace=True, usecols=fields)
#sheet_name ='Page 1',
print(df.shape)

(32000, 2)


In [80]:
xlsx = pd.ExcelFile('user_incidents_dumps_60days.xlsx')
fields = ['Short description', 'Assignment group']
#df = pd.read_excel('user_incidents_dumps_60days.xlsx', skipinitialspace=True, usecols=fields)
data_sheets = []
for sheet in xlsx.sheet_names:
    data_sheets.append(pd.read_excel(xlsx,sheet,skipinitialspace=True, usecols=fields))
df = pd.concat(data_sheets)



In [82]:
df.columns = [c.replace(' ', '_') for c in df.columns]

In [83]:
df.head()

,Short_description,Assignment_group
0,Network Account || Unable to login || Account ...,DH-Enterprise IT Service Cntr
1,Ms4 - phxasp01 - job needs to be killed / job...,DH-Enterprise IT Service Cntr
2,Printer || Unable to print the strips through...,DH-NC-EUS Stockton
3,Clairiva - unable to login / user id: ltimpog ...,DH-Enterprise IT Service Cntr
4,WOW || Unable to turn on || Black Screen || De...,DH-SC-EUS Bakersfield MSH


In [100]:
#get data with at leaset count(assignment group)>200
df1=df.groupby("Assignment_group").filter(lambda x: len(x) > 500)
print(df1.shape)

(43822, 2)


In [101]:
#print count of assignment groups
df1.Assignment_group.value_counts()

DH-Enterprise IT Service Cntr        27828
DH-Helpdesk HDAG                      1855
DHE-HR Tier 2 - Talent Management     1612
DH-SW-EUS StJoseph                    1311
DH-Helpdesk RRE                       1226
DH-MPS Kyocera                        1058
DH-ClinApps NAS                       1047
DH-SW-EUS Chandler                     956
DH-NC-EUS Stockton                     765
DH-SC-EUS StJohnRMC                    673
DH-NC-Telcom Sacramento                624
DH-SW-EUS Phoenix                      593
DH-SC-EUS NLA NrthrdgRoscoe            571
DH-NC-EUS Redding                      571
DH-GB-EUS Dominican                    546
DH-ClinApps CPOE                       545
DHE-SecAdmin                           526
DH-NC-EUS Sac MSJH                     508
DH-SC-EUS Bakersfield BMH              504
DH-NC-EUS Sac MGH                      503
Name: Assignment_group, dtype: int64

In [102]:
#No. of unique assignment groups =  num of output classes
num_classes = df1.Assignment_group.nunique()
print(num_classes)

20


In [99]:
#save data to pickle format
df1.to_pickle("snow_v1_43k_20out_0910.pkl")
#df = pd.read_pickle("snow_v1_dataframe.pkl")

In [103]:
#Create input and output data sets
X = df1.Short_description
Y = df1.Assignment_group
print(X.shape,Y.shape)

(43822,) (43822,)


In [104]:
#Encode outputs into N- dim one hot encoded matrix
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#print(onehot_encoded)
# invert first example to return original, must for return api --how to inverse transform post deploy
#inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
#print(inverted)

In [105]:
print(onehot_encoded.shape)

(43822, 20)


In [90]:
df_col_len = int(df1['Short_description'].str.split().str.len().max())
print(df_col_len)


35


In [106]:
#Split into train and test(15%)
X_train,X_test,Y_train,Y_test = train_test_split(X,onehot_encoded,test_size=0.15)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(37248,) (6574,) (37248, 20) (6574, 20)


In [107]:
# Tokenize and pad , init max sizes...should find way to eff value max words 
max_words = 5000
max_len = 20 #df_col_len #35
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
print(sequences_matrix.shape)

(37248, 20)


In [108]:
#define LSTM  model
#tweak parameters/layers to inc. eff, smtimes 1D conv also used
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs) #50 dim
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(num_classes,name='out_layer')(layer)  #num_classes=# of outputs
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [109]:
#compile model with optimizer -  can use RMSProp or Adam
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 20)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 20)                5140      
__________

In [110]:
#execute the model, early stopping if model stops converging, batch size can be tweaked. 20% data for validation each epoch
history = model.fit(sequences_matrix,Y_train,batch_size=64,epochs=15,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 29798 samples, validate on 7450 samples
Epoch 1/15
29798/29798 [==============================] - 19s 627us/step - loss: 1.2043 - acc: 0.6868 - val_loss: 0.9815 - val_acc: 0.7470
Epoch 2/15
29798/29798 [==============================] - 17s 564us/step - loss: 0.9045 - acc: 0.7602 - val_loss: 0.8615 - val_acc: 0.7664
Epoch 3/15
29798/29798 [==============================] - 17s 554us/step - loss: 0.8156 - acc: 0.7791 - val_loss: 0.8156 - val_acc: 0.7808
Epoch 4/15
29798/29798 [==============================] - 17s 571us/step - loss: 0.7675 - acc: 0.7887 - val_loss: 0.7874 - val_acc: 0.7883
Epoch 5/15
29798/29798 [==============================] - 17s 576us/step - loss: 0.7351 - acc: 0.7961 - val_loss: 0.7743 - val_acc: 0.7902
Epoch 6/15
29798/29798 [==============================] - 17s 561us/step - loss: 0.7087 - acc: 0.8052 - val_loss: 0.7719 - val_acc: 0.7862
Epoch 7/15
29798/29798 [==============================] - 16s 533us/step - loss: 0.6851 - acc: 0.8113 - val_loss: 0.7

In [111]:
#save model to file
model.save('model_snow_v1_43k_20out_0910.h5')

In [ ]:
##notes
# input=24k, output classes=27,BS=64,RMS,LSTM(64,256,),dropout 0.5,epochs=9, eff=75%; if BS=128,epoch=2,eff same
# input=55kk,output classes=57,BS=64,RMS,LSTM(64,256,),drop 0.5,epoch 5, eff=64%
# input=43k,out classes=20,BS=64,RMS,LSTM(64,256,),drop 0.5,epoch 9, eff=80%

In [112]:
#Create sequences for test data
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [113]:
#Calculate accuracy on test data
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3%}'.format(accr[0],accr[1]))

6574/6574 [==============================] - 1s 181us/step
Test set
  Loss: 0.750
  Accuracy: 80.894%


In [47]:
#load unseen data for validation
df_val = read_excel('validate2018.xlsx', sheet_name = 'Sheet1')
Xnew=df_val.Description
Ynew_orig = df_val.Group
print(Xnew.shape,Ynew_orig.shape)

(13, 2)


In [115]:
#convert unseen data to word tokens
Xnew_2 = sequence.pad_sequences(tok.texts_to_sequences(Xnew),maxlen=max_len)

In [116]:
#predict output on unseen data
ynew = model.predict(Xnew_2)

In [117]:
#Print unseen data, predicted value, original value
for i in range(len(Xnew)):
	print("X=%s,\nPredicted=%s,\nOriginal=%s\n" % (Xnew[i], label_encoder.inverse_transform([argmax(ynew[i, :])]),Ynew_orig[i]))

X=zOther Clinical Application Issue || Helpdesk - Clinical Application Issue,
Predicted=['DH-Helpdesk HDAG'],
Original=DH-Helpdesk HDAG

X=Monitor ||  How to increase the brightness || ,
Predicted=['DH-Enterprise IT Service Cntr'],
Original=DH-Enterprise IT Service Cntr

X=Outlook || Outlook Application Issue,
Predicted=['DH-Helpdesk RRE'],
Original=DH-Helpdesk RRE

X=Phone || Phone Issue,
Predicted=['DH-NC-Telcom Sacramento'],
Original=DH-NC-Telcom Sacramento

X=Kyocera Printer Issue || Kyocera Printer Issue,
Predicted=['DH-MPS Kyocera'],
Original=DH-MPS Kyocera

X=Emergency Account Disablement || user ID : aguise001 || Need to be disabled Immediately ,
Predicted=['DHE-SecAdmin'],
Original=DHE-SecAdmin

X=Network, account lock, username : jfunk002,
Predicted=['DH-Enterprise IT Service Cntr'],
Original=DH-Enterprise IT Service Cntr

X=printer || patient data not printing up from cerner ,
Predicted=['DH-ClinApps NAS'],
Original=DH-ClinApps HIM

X=network account  || password reset || us